In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
address="C:\\Users\\Amirhossein\\Desktop\\Fuzzy BWM.xlsx"

df=pd.read_excel(address, sheet_name = "Criteria", skiprows = 2,  nrows= 5, usecols=[1,2,3],header=None)
df.columns=["Criteria","Best","Worst"]
print(df)
Best=df["Criteria"][df[df['Best'] == "Equally importance"].index.tolist()[0]]
Worst=df["Criteria"][df[df['Worst'] == "Equally importance"].index.tolist()[0]]
df.columns=["Criteria",Best,Worst]
Cnum=df.shape[0]

df.set_index(df["Criteria"],inplace=True)
# Fuzzification
Fuzzy={"Equally importance": [1, 1, 1],
        "Weakly important": [2/3, 1, 3/2],
        "Fairly important":[3/2, 2, 5/2],
        "Very important":[5/2, 3, 7/2],
        "Absolutely important":[7/2, 4, 9/2]}

for i in [Best,Worst]:
    for j in range(df.shape[0]):
        df[i][j]=Fuzzy[df[i][j]]
        
weight=dict()

     Criteria                Best                 Worst
0  Criteran 1    Fairly important  Absolutely important
1  Criteran 2  Equally importance        Very important
2  Criteran 3      Very important    Equally importance
3  Criteran 4      Very important        Very important
4  Criteran 5    Fairly important      Fairly important


In [11]:
#=============================================================================
# Model
#=============================================================================
import pyomo.environ as pyo
Model = pyo.ConcreteModel()

# Set
Model.Criteria=pyo.Set(initialize=df["Criteria"])

# Set and variable
Model.L=pyo.Var(Model.Criteria, within=pyo.PositiveReals)
Model.M=pyo.Var(Model.Criteria, within=pyo.PositiveReals)
Model.U=pyo.Var(Model.Criteria, within=pyo.PositiveReals)
Model.ksi=pyo.Var(within=pyo.NonNegativeReals)

def obj1(model):
    return Model.ksi
Model.obj1=pyo.Objective(expr=obj1 ,sense=pyo.minimize)
# Model.obj1.pprint()

L=[Model.L, Model.M, Model.U]
Lp=L.copy(); Lp.reverse()

def Co1(Model, i, j):
    return (j[Best]-df[Best][i][L.index(j)]*Lp[L.index(j)][i]<= Model.ksi*Lp[L.index(j)][i])
Model.Co1 = pyo.Constraint(Model.Criteria, L, rule=Co1)
# Model.Co1.pprint()

def Co2(Model, i, j):
    return (j[Best]-df[Best][i][L.index(j)]*Lp[L.index(j)][i]>= -Model.ksi*Lp[L.index(j)][i])
Model.Co2 = pyo.Constraint(Model.Criteria, L, rule=Co2)
# Model.Co2.pprint()

def Co3(Model, i, j):
    return (j[i]-df[Worst][i][L.index(j)]*Lp[L.index(j)][Worst]<= Model.ksi*Lp[L.index(j)][Worst])
Model.Co3 = pyo.Constraint(Model.Criteria, L, rule=Co3)
# Model.Co3.pprint()

def Co4(Model, i,j):
    return (j[i]-df[Worst][i][L.index(j)]*Lp[L.index(j)][Worst]>= -Model.ksi*Lp[L.index(j)][Worst])
Model.Co4 = pyo.Constraint(Model.Criteria, L, rule=Co4)
# Model.Co4.pprint()

def Co5(Model):
    return (sum(j[i] if L.index(j)!=1 else 4*j[i] for i in Model.Criteria for j in L)/6==1)
Model.Co5 = pyo.Constraint(rule=Co5)
# Model.Co5.pprint()

def Co6(Model,i):
    return (Model.L[i]<=Model.M[i])
Model.Co6 = pyo.Constraint(Model.Criteria,rule=Co6)
# Model.Co6.pprint()

def Co7(Model,i):
    return (Model.M[i]<=Model.U[i])
Model.Co7 = pyo.Constraint(Model.Criteria,rule=Co7)

In [12]:
import pyomo.opt as pyopt
results=pyopt.SolverFactory("ipopt").solve(Model, tee=True) # doctest: +SKIP

for i in Model.Criteria:
    a=[]
    for j in L:
        a.append(j[i].value)
    weight.update({i: a})
    
"""
print("ksi= ",Model.ksi())
Model.pprint()
Model.display()
        
"""

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:       15
Number of nonzeros in inequality constraint Jacobian.:      196
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:       16
                     variables with only lower bounds:       16
                variables with lower and upper bounds:        0


'\nprint("ksi= ",Model.ksi())\nModel.pprint()\nModel.display()\n        \n'

In [13]:
weight

{'Criteran 1': [0.22232570861069512, 0.2660321473360896, 0.363098767656071],
 'Criteran 2': [0.2970257163652376, 0.3163762940980174, 0.3411552877341889],
 'Criteran 3': [0.07726211680920207, 0.07909407332180773, 0.08710677608079348],
 'Criteran 4': [0.13344002507080333, 0.15818814434631215, 0.19566531895717929],
 'Criteran 5': [0.12555906181771817, 0.16170821126929297, 0.231765739412031]}